# Notebook ICD - 13


In [1]:
import numpy as np
import pandas as pd
import math
from collections import Counter


## ID3 desde cero

ID3 (Iterative Dichotomiser 3) fue desarrollado en 1986 por Ross Quinlan. El algoritmo crea un árbol de decisiones de múltiples vías, encontrando para cada nodo (es decir, de manera voraz) la característica categórica que producirá la mayor ganancia de información para objetivos categóricos. Los árboles se expanden hasta su tamaño máximo y luego se aplica un paso de poda para mejorar la capacidad del árbol de generalizar a datos no vistos.

C4.5 es el sucesor de ID3 y eliminó la restricción de que las características debían ser categóricas al definir dinámicamente un atributo discreto (basado en variables numéricas) que particiona el valor continuo del atributo en un conjunto discreto de intervalos. C4.5 convierte los árboles entrenados (es decir, el resultado del algoritmo ID3) en conjuntos de reglas if-then (si-entonces). Luego se evalúa la precisión de cada regla para determinar el orden en que deben aplicarse. La poda se realiza eliminando la condición previa de una regla si la precisión de la regla mejora sin ella.

C5.0 es la última versión de Quinlan, lanzada bajo una licencia propietaria. Utiliza menos memoria y construye conjuntos de reglas más pequeños que C4.5, a la vez que es más preciso.

## Estimación de Entropía

La entropía mide la incertidumbre de un conjunto de datos. En esta función, se cuenta la frecuencia de cada clase y luego se utiliza la fórmula de entropía para calcular la cantidad de incertidumbre en las etiquetas de clase. La entropía alcanza su valor mínimo (cero) cuando todas las instancias pertenecen a una sola clase. Por el contrario, la entropía máxima (uno) se obtiene cuando hay una distribución perfectamente equilibrada entre las clases.

In [2]:
def entropy(y):
    # Cuenta la frecuencia de una variable
    counter = Counter(y)
    total = len(y)
    
    # Calcula la entropía
    ent = 0.0
    for count in counter.values():
        probability = count / total
        ent -= probability * math.log2(probability)
    return ent

# Calcular la ganancia de información

La ganancia de información se basa en la disminución de la entropía al dividir los datos según un atributo. 
La función primero calcula la entropía total del conjunto de etiquetas y luego resta la entropía ponderada de cada subconjunto, lo que nos da la ganancia de información.

In [4]:
def information_gain(X_column, y):
    # Entropía del conjunto original
    ent_total = entropy(y)
    
    # Dividir los datos por el valor de la característica
    values, counts = np.unique(X_column, return_counts=True)
    
    # Entropía ponderada de los subconjuntos
    weighted_entropy = 0.0
    for value, count in zip(values, counts):
        subset_y = y[X_column == value]
        if len(subset_y) == 0:  # Verificamos si el subconjunto está vacío
            continue
        weighted_entropy += (count / len(y)) * entropy(subset_y)
    
    # Ganancia de información
    return ent_total - weighted_entropy

# Seleccionar el mejor atributo

Aquí iteramos sobre cada columna (atributo) en el conjunto de datos para calcular la ganancia de información. El atributo con la mayor ganancia de información se selecciona para realizar la partición

In [5]:
def best_attribute(X, y):
    best_gain = -1
    best_attr = None
    
    for col in X.columns:
        gain = information_gain(X[col], y)
        if gain > best_gain:
            best_gain = gain
            best_attr = col
    return best_attr

# Construir el árbol de decisión

Esta es la implementación central del algoritmo ID3. Si todas las etiquetas son iguales, se crea un nodo hoja con la clase correspondiente. Si no quedan atributos, se asigna la clase mayoritaria. Se selecciona el mejor atributo con la mayor ganancia de información y se crea el nodo raíz. Luego, el conjunto de datos se divide y se construyen subárboles de manera recursiva.

In [6]:
class Node:
    def __init__(self, feature=None, value=None, children=None, *, label=None):
        self.feature = feature  # El nombre del atributo
        self.value = value  # El valor del atributo
        self.children = children if children is not None else {}  # Subárboles
        self.label = label  # Clase terminal

def id3(X, y):
    if len(np.unique(y)) == 1:  # Si todas las etiquetas son iguales
        return Node(label=np.unique(y)[0])
    
    if X.empty:  # Si no hay más atributos
        return Node(label=Counter(y).most_common(1)[0][0])  # Retornar la clase mayoritaria
    
    # Seleccionamos el mejor atributo
    best_attr = best_attribute(X, y)
    
    # Creamos el nodo
    root = Node(feature=best_attr)
    
    # Para cada valor posible del mejor atributo, creamos un subárbol
    values = np.unique(X[best_attr])
    for value in values:
        subset_X = X[X[best_attr] == value].drop(columns=[best_attr])
        subset_y = y[X[best_attr] == value]
        
        if len(subset_X) == 0:
            continue
        
        # Llamada recursiva a id3
        subtree = id3(subset_X, subset_y)
        root.children[value] = subtree
    
    return root


# Predicción (inferencia) con el árbol aprendido

Para hacer predicciones, el árbol de decisión se recorre de manera recursiva. Al llegar a un nodo hoja, se devuelve la etiqueta asociada. En cada nodo interno, la decisión se toma en función del valor del atributo.

In [7]:
def predict(tree, X_test):
    if tree.label is not None:  # Nodo hoja
        return tree.label
    
    value = X_test[tree.feature]  
    if value in tree.children:
        return predict(tree.children[value], X_test)
    else:
        return None

# Ejemplo de implementación

El algoritmo ID3 se aplicará al conjunto de datos 'Play Tennis' para construir un árbol de decisión que prediga si jugar al tenis o no, basado en condiciones meteorológicas como la temperatura, la humedad y el viento. Las 14 instancias disponibles servirán como base de entrenamiento para el modelo, mientras que una nueva instancia, no incluida en el entrenamiento, se usará para evaluar su rendimiento y capacidad de generalización.

In [8]:
data = pd.read_csv('weather.nominal.csv')

# Definir X (características) e y (etiqueta)
X = data.iloc[:, :-1]  # Todas las columnas menos la última
y = data.iloc[:, -1]  # Última columna (etiqueta)

# Entrenar el árbol de decisión usando el algoritmo ID3 con los nombres originales de las columnas
tree = id3(X, y)

# Crear la instancia para probar: sunny, hot, normal, TRUE
test_instance = {'outlook': 'sunny', 'temperature': 'hot', 'humidity': 'normal', 'windy': True}

# Realizar la predicción
prediction = predict(tree, test_instance)
print(f'Predicción para la instancia {test_instance}: {prediction}')

Predicción para la instancia {'outlook': 'sunny', 'temperature': 'hot', 'humidity': 'normal', 'windy': True}: yes



# Scikit-learn implementation

CART (Classification and Regression Trees) is very similar to C4.5, but it differs in that it supports numerical target variables (regression) and does not compute rule sets. CART constructs binary trees using the feature and threshold that yield the largest information gain at each node. Scikit-learn uses an optimized version of the CART algorithm.

## Library

In [9]:
from sklearn.tree import DecisionTreeClassifier

In [11]:
df = pd.read_csv(r'weather.nominal.csv')

In [12]:
print(df)

     outlook temperature humidity  windy play
0      sunny         hot     high  False   no
1      sunny         hot     high   True   no
2   overcast         hot     high  False  yes
3      rainy        mild     high  False  yes
4      rainy        cool   normal  False  yes
5      rainy        cool   normal   True   no
6   overcast        cool   normal   True  yes
7      sunny        mild     high  False   no
8      sunny        cool   normal  False  yes
9      rainy        mild   normal  False  yes
10     sunny        mild   normal   True  yes
11  overcast        mild     high   True  yes
12  overcast         hot   normal  False  yes
13     rainy        mild     high   True   no


In [14]:
# defining the dependent and independent variables
X_train = df[['outlook', 'temperature', 'humidity', 'windy']]
y_train = df[['play']]

print(X_train.head())
print(y_train.head())


    outlook temperature humidity  windy
0     sunny         hot     high  False
1     sunny         hot     high   True
2  overcast         hot     high  False
3     rainy        mild     high  False
4     rainy        cool   normal  False
  play
0   no
1   no
2  yes
3  yes
4  yes


# De categórico a numérico

Scikit-learn utiliza una versión optimizada del algoritmo CART; sin embargo, la implementación de scikit-learn no admite variables categóricas por el momento.

In [33]:


from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

outlook = X_train.iloc[:,0]
outlook_enc = encoder.fit_transform(outlook)
print(outlook.tolist())
print(outlook_enc)

temperature = X_train.iloc[:,1]
temperature_enc = encoder.fit_transform(temperature)
print(temperature.tolist())
print(temperature_enc)

humidity = X_train.iloc[:,2]
humidity_enc = encoder.fit_transform(humidity)
print(humidity.tolist())
print(humidity_enc)

wind = X_train.iloc[:,3]
wind_enc = encoder.fit_transform(wind)
print(wind.tolist())
print(wind_enc)



['sunny', 'sunny', 'overcast', 'rainy', 'rainy', 'rainy', 'overcast', 'sunny', 'sunny', 'rainy', 'sunny', 'overcast', 'overcast', 'rainy']
[2 2 0 1 1 1 0 2 2 1 2 0 0 1]
['hot', 'hot', 'hot', 'mild', 'cool', 'cool', 'cool', 'mild', 'cool', 'mild', 'mild', 'mild', 'hot', 'mild']
[1 1 1 2 0 0 0 2 0 2 2 2 1 2]
['high', 'high', 'high', 'high', 'normal', 'normal', 'normal', 'high', 'normal', 'normal', 'normal', 'high', 'normal', 'high']
[0 0 0 0 1 1 1 0 1 1 1 0 1 0]
[False, True, False, False, False, True, True, False, False, False, True, True, False, True]
[0 1 0 0 0 1 1 0 0 0 1 1 0 1]


In [34]:
df_outlook = pd.DataFrame(outlook_enc, columns = ['Outlook'])

#df_temperature = pd.DataFrame(outlook_enc, columns = ['temperature'])
# df_humidity = pd.DataFrame(outlook_enc, columns = ['humidity'])
df_wind = pd.DataFrame(outlook_enc, columns = ['Wind'])
X_train_num = pd.concat([df_outlook, X_train.iloc[:,1], X_train.iloc[:,3], df_wind], axis=1)
print(X_train_num)

    Outlook temperature  windy  Wind
0         2         hot  False     2
1         2         hot   True     2
2         0         hot  False     0
3         1        mild  False     1
4         1        cool  False     1
5         1        cool   True     1
6         0        cool   True     0
7         2        mild  False     2
8         2        cool  False     2
9         1        mild  False     1
10        2        mild   True     2
11        0        mild   True     0
12        0         hot  False     0
13        1        mild   True     1


# Build the decision tree

In [35]:
clf = DecisionTreeClassifier().fit(X_train_num, y_train)

ValueError: could not convert string to float: 'hot'

In [22]:
from sklearn import tree
features = X_train_num.columns.values.tolist()
text_representation = tree.export_text(clf, feature_names = features)
print(text_representation)

NameError: name 'clf' is not defined